In [1]:
# !pip install kagglehub

In [2]:
# import kagglehub

# # Download latest version from Kaggle using dataset handle
# path = kagglehub.dataset_download("spsayakpaul/arxiv-paper-abstracts")

# print("Path to dataset files:", path)

In [3]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("ivanmitriakhin/arxiv-titles-abstracts-and-tags")

# print("Path to dataset files:", path)

In [4]:
# !pip install transformers[torch]==4.38.0 datasets pandas scikit-learn tabulate accelerate==0.27.2


In [5]:
# !pip install "numpy<2.0"

In [6]:
import pandas as pd
import torch
from datasets import Dataset
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import re
import transformers
import accelerate

/mnt/c/Users/User/Desktop/bert/12venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# --- 1. Configuration ---
file_path = 'arxiv_data_grouped.csv'
title_column = 'titles'
abstract_column = 'abstracts'
MODEL_CHECKPOINT = 'allenai/scibert_scivocab_uncased'
output_model_dir = './fine-tuned-scibert-multilabel'

In [8]:
# --- 2. Data Loading and Cleaning ---
print(f"Loading data from: {file_path}")
df = pd.read_csv(file_path)

def clean_text(text):
    if not isinstance(text, str): 
        return ""
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

print("Cleaning text data...")
df[title_column] = df[title_column].apply(clean_text)
df[abstract_column] = df[abstract_column].apply(clean_text)

Loading data from: arxiv_data_grouped.csv
Cleaning text data...


In [9]:
# --- 3. Data Preprocessing ---
first_label_index = df.columns.get_loc(abstract_column) + 1
label_columns = df.columns[first_label_index:].tolist()
df['text'] = df[title_column] + " [SEP] " + df[abstract_column]

# Convert labels to float32 numpy arrays (this ensures proper dtype)
df['labels'] = df[label_columns].values.astype(np.float32).tolist()
df_clean = df[['text', 'labels']]

print(f"Dataset shape: {df_clean.shape}")
print(f"Number of label classes: {len(label_columns)}")
print(f"Sample labels: {df_clean['labels'].iloc[0]}")

Dataset shape: (536914, 2)
Number of label classes: 8
Sample labels: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [10]:
# --- 4. Tokenization ---
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)


Loading tokenizer...


/mnt/c/Users/User/Desktop/bert/12venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# --- 5. Dataset Creation and Processing ---
print("Creating dataset...")
full_dataset = Dataset.from_pandas(df_clean)

print("Tokenizing dataset...")
tokenized_dataset = full_dataset.map(tokenize_function, batched=True)

# Remove text column (no longer needed)
tokenized_dataset = tokenized_dataset.remove_columns(['text'])

# Set format to torch tensors (this handles the float32 conversion automatically)
print("Setting dataset format to PyTorch tensors...")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Verify the conversion worked
sample = tokenized_dataset[0]
print(f"Labels type: {type(sample['labels'])}")
print(f"Labels dtype: {sample['labels'].dtype}")
print(f"Sample labels: {sample['labels']}")

Creating dataset...
Tokenizing dataset...


Map: 100%|██████████| 536914/536914 [01:43<00:00, 5189.05 examples/s]


Setting dataset format to PyTorch tensors...
Labels type: <class 'torch.Tensor'>
Labels dtype: torch.float32
Sample labels: tensor([1., 0., 0., 0., 0., 0., 0., 1.])


In [12]:
# --- 6. Train/Test Split ---
print("Splitting dataset...")
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")

Splitting dataset...
Train dataset size: 429531
Eval dataset size: 107383


In [13]:
# --- 7. Model Loading ---
print("Loading model...")
num_labels = len(label_columns)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    problem_type="multi_label_classification",
    num_labels=num_labels
)

Loading model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# --- 8. Metrics Function ---
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # Convert to probabilities
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(logits))
    
    # Convert to predictions (threshold = 0.5)
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= 0.5)] = 1
    
    # Calculate metrics
    f1_micro = f1_score(labels, predictions, average='micro')
    f1_macro = f1_score(labels, predictions, average='macro')
    roc_auc = roc_auc_score(labels, predictions, average='micro')
    accuracy = accuracy_score(labels, predictions)
    
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'roc_auc': roc_auc,
        'accuracy': accuracy
    }

In [15]:
# --- 9. Training Arguments ---
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=128,    
    per_device_eval_batch_size=128,
    
    # Add these optimizations:
    fp16=True,                         # Half precision (faster + less memory)
    dataloader_num_workers=2,          # Parallel data loading
    dataloader_pin_memory=True,        # Faster transfer
    gradient_checkpointing=True,       # Save memory
    
    # Existing settings...
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
)

In [16]:
print(f"VERIFICATION - Batch size: {training_args.per_device_train_batch_size}")
print(f"Expected steps per epoch: {len(train_dataset) // training_args.per_device_train_batch_size}")

VERIFICATION - Batch size: 128
Expected steps per epoch: 3355


In [17]:
print(f"Training dataset size: {len(train_dataset)}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Gradient accumulation steps: {training_args.gradient_accumulation_steps}")
print(f"Expected calculation: {len(train_dataset)} ÷ {training_args.per_device_train_batch_size} = {len(train_dataset) // training_args.per_device_train_batch_size}")

Training dataset size: 429531
Batch size: 128
Gradient accumulation steps: 1
Expected calculation: 429531 ÷ 128 = 3355


In [ ]:
# --- 10. Trainer Initialization and Training ---
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Add early stopping callback

)

print("STARTING MODEL TRAINING")

# Train the model
trainer.train()

print("TRAINING COMPLETE")


STARTING MODEL TRAINING


/mnt/c/Users/User/Desktop/bert/12venv/lib/python3.12/site-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/mnt/c/Users/User/Desktop/bert/12venv/lib/python3.12/site-packages/torch/_d

Epoch,Training Loss,Validation Loss


In [ ]:
# Save the Model 
print(f"Saving model to: {output_model_dir}")
trainer.save_model(output_model_dir)
tokenizer.save_pretrained(output_model_dir)


In [ ]:
# Final Evaluation
print("\nRunning final evaluation...")
eval_results = trainer.evaluate()

print("\nFinal Evaluation Results:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

print(f"\nModel saved successfully to: {output_model_dir}")
print("Training pipeline completed!")